In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from sklearn.preprocessing import LabelEncoder
#from wordcloud import WordCloud
from textblob import TextBlob
#from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.impute import KNNImputer

from numpy.linalg import norm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.compose import ColumnTransformer


In [3]:
Dataset = pd.read_csv("C:/Users/hmq4/OneDrive/Desktop/Data Science- Misk/chrome/Misk_DSI_Capstone_Project/Final_dataset.csv")

In [5]:
Dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3739 entries, 0 to 3738
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        3739 non-null   int64 
 1   Name              3739 non-null   object
 2   Price             3739 non-null   int64 
 3   Description       3739 non-null   object
 4   Rate              3739 non-null   object
 5   Rating_count      3739 non-null   object
 6   image             3739 non-null   object
 7   Brand             3739 non-null   object
 8   Gender            3739 non-null   object
 9   Product_Type      3739 non-null   object
 10  Character_x       3739 non-null   object
 11  Fragrance_Family  3682 non-null   object
 12  Size              3739 non-null   object
 13  Year              3739 non-null   object
 14  Ingredients       3739 non-null   object
 15  Concentration     3739 non-null   object
 16  Top_note          3739 non-null   object
 17  Middle_note   

In [4]:
Dataset.shape

(4090, 19)

In [5]:
Dataset.isnull().sum()

Unnamed: 0           0
Name                 0
Price                0
Description          0
Rate                 0
Rating_count         0
image                0
Brand               18
Gender               5
Product_Type        27
Character_x          2
Fragrance_Family    59
Size                 1
Year                 4
Ingredients          3
Concentration       12
Top_note             1
Middle_note          1
Base_note           29
dtype: int64

In [6]:
Dataset = Dataset.drop("Unnamed: 0", axis=1)

- Reducing the Description 

In [7]:
pd.set_option("display.max_colwidth", -1)
Dataset.Description.head(1)

C:\Users\hmq4\AppData\Local\Temp\ipykernel_15304\2736223612.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option("display.max_colwidth", -1)


0    Perfume for the energetic woman who is a hero in her movie in life every day! It keeps you vibrant and sparkling with irresistible attractiveness, a combination of kiwi, lemon and sandalwood.\nAbout the brand:\nDolce & Gabbana was established in 1985 by two Italian fashion designers. Dolce & Gabbana has not only managed to build a loyal customer base in such a short span of time but it also partnered with Motorola and Sony Ericson for multiple campaigns. D&G has also partnered with football clubs namely AC Milan and Chelsea FC.
Name: Description, dtype: object

In [8]:
#df.Description.split("About the brand:", 1)
Dataset['Description'] = Dataset['Description'].str.split("\nAbout the brand:\n").str[0]

In [9]:
pd.set_option("display.max_colwidth", -1)
Dataset.Description.head(1)

C:\Users\hmq4\AppData\Local\Temp\ipykernel_15304\2736223612.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option("display.max_colwidth", -1)


0    Perfume for the energetic woman who is a hero in her movie in life every day! It keeps you vibrant and sparkling with irresistible attractiveness, a combination of kiwi, lemon and sandalwood.
Name: Description, dtype: object

In [11]:
Dataset.head()

,Name,Price,Description,Rate,Rating_count,image,Brand,Gender,Product_Type,Character_x,Fragrance_Family,Size,Year,Ingredients,Concentration,Top_note,Middle_note,Base_note
0,Dolce & Gabanna L'imperatrice 3 Pour Femme,199,"Perfume for the energetic woman who is a hero in her movie in life every day! It keeps you vibrant and sparkling with irresistible attractiveness, a combination of kiwi, lemon and sandalwood.",5,6 Rating,https://assets.goldenscent.com/catalog/product/cache/1/image/9df78eab33525d08d6e5fb8d27136e95/3/4/3423473020615-dolce-_-gabbana-l_imperatrice-01_1_.png,Dolce&Gabbana,Women,Perfume,Romantic,Floral,100 ml,2009,"Watermerlon, Kiwi, Pink Cyclamen, Musk, Pink Pepper, Jasmine, Sandalwood, Lemon Tree",Eau de Toilette,"Pink pepper, kiwi, rhubarb","jasmine, cyclamen, watermelon","musk, sandalwood, lemon trees."
1,Roberto Cavalli Paradiso,169,"Woody floral fragrance, a subtle aroma that makes you feel fresh with a stunning blend of citrus, jasmine, and cypress.",4.95,17 Rating,https://assets.goldenscent.com/catalog/product/cache/1/image/9df78eab33525d08d6e5fb8d27136e95/3/6/3607347733423c-roberto-cavalli-paradiso---75-ml.jpg,Roberto Cavalli,Women,Perfume,Romantic,Woody,50 ml,2015,"Citrus, mandarin, bergamot, jasmine, pine, cypress, laurel.",Eau de Parfum,"Citruses , Mandarin , Bergamot",Jasmine,"Cypress, Parasol pine, Pink laurel"
2,Yves Saint Laurent Libre,389,"This perfume is a reflection of Freedom, specially designed for bold women, for those who live by their own rules, to enjoy the unique fusion of French lavender & orange blossoms from Morrocco. A statement of freedom with an additional whiff of daring musky notes.",5,3 Rating,https://assets.goldenscent.com/catalog/product/cache/1/image/9df78eab33525d08d6e5fb8d27136e95/3/6/3614272648418c-yves-saint-laurent-libre---90-ml_1_.jpg,Yves Saint Laurent,Women,Perfume,Romantic,Floral,90 ml,2019,"Mandarin Orange, lavendar, black currant, petitgrain, jasmine, orange blossom, vanilla, cedar, musk, ambergris",Eau de Parfum,"Mandarin Orange, Lavendar, Black Currant, Petitgrain","Jasmine, Orange Blossom","Vanilla, Cedar, Musk, Ambergris"
3,Mancera Red Tobacco,499,"Mancera Red Tobacco is the new oriental, woody fragrance for men and women. Its the best oriental scent, full of life and attractiveness. This perfume contains a very high quality ingredients. A must have perfume!:",4.38,8 Rating,https://assets.goldenscent.com/catalog/product/cache/1/image/9df78eab33525d08d6e5fb8d27136e95/3/6/3607347733423c-mancera-red-tobacco---120-ml.jpg,Mancera,Women,Perfume,Romantic,Oriental,120 ml,2017,"Saffron, Cinnamon, Incense, Nutmeg, White Peach, Green Apple & Nepalese Oud, Leaves of Patchouli, delicate Jasmine, Precious Tobacco, Amber, Woody Notes, Vetiver, Vanilla Pods, White Musk.",Eau de Parfum,"Saffron, Cinnamon, Incense, Nutmeg, White peach, Green apple, Nepalese","Patchouli, Jasmine","Tobacco, Amber, Woody notes, Vetiver, Vanilla, White musk"
4,Giorgio Armani Emporio Armani Stronger With You Intensely,399,"for every romantic gentle man, This strong fascinating fragrance is for you. It was released to capture all people hearts from the first smell to tell a story of an outstanding emotional man who is deeply in love and is ready to do anything for their lovers.",5,3 Rating,https://assets.goldenscent.com/catalog/product/cache/1/image/9df78eab33525d08d6e5fb8d27136e95/3/6/3614272225718-giorgio-armani-emporio-armani-stronger-with-you-intensely.jpg,Giorgio Armani,Men,Perfume,Romantic,Aromatic,100 ml,2019,"Spices, Violet, Lavender, Sweet Toffee, Caramel, Cinnamon, Suede, Vanilla, Amber",Eau de Parfum,"Pink pepper, Juniper, Violet leaf","Lavender, Sage, Toffee, Cinnamon","Tonka bean, Suede, Amber, Vanilla"


In [13]:
#Dataset.groupby("Fragrance_Family").agg({'Price' :"mean"}).sort_values(by = "Price", ascending=False)

In [ ]:
#df['Time'] = df['Time'].str.lower()
#df['Flavor'] = df['Flavor'].str.lower()
#df['Ingredients'] = df['Ingredients'].str.lower()
#df['Color'] = df['Color'].str.lower()
#df['Description'] = df['Description'].str.lower()
#df['Type'] = df['Type'].str.lower()
#df['Ingredients'] = df['HealthP'].str.lower()

In [ ]:
#df['word_count'] = df['Ingredients'].apply(lambda x: len(str(x).split()))
#df['word_count'].plot(
#    kind='hist',
#    bins = 50,
#    figsize = (12,8),title='Tea Ingredients Word Count Distribution')

In [ ]:
#df['word_count'] = df['Description'].apply(lambda x: len(str(x).split()))# Plotting the word count
#df['word_count'].plot(
#    kind='hist',
#    bins = 50,
#    figsize = (12,8),title='Word Count Distribution for tea descriptions')

### Label Encoding 

In [ ]:
le = LabelEncoder()
le.fit(df.Character_x)
df.Character_x=le.transform(df.Character_x)

le.fit(df.Fragrance_Family)
df.Fragrance_Family=le.transform(df.Fragrance_Family)

le.fit(df.Ingredients)
df.Ingredients=le.transform(df.Ingredients)

le.fit(df.Concentration)
df.Concentration=le.transform(df.Concentration)

le.fit(df.Top_note)
df.Top_note=le.transform(df.Top_note)

le.fit(df.Middle_note)
df.Middle_note=le.transform(df.Middle_note)

le.fit(df.Base_note)
df.Base_note=le.transform(df.Base_note)

le.fit(df.Gender)
df.Gender=le.transform(df.Gender)


In [ ]:
features_df = df[[
 'Gender',
 'Character_x',
 'Fragrance_Family',
 'Ingredients',
 'Concentration',
 'Top_note',
 'Middle_note',
 'Base_note']]

In [ ]:
# Finding the correlation between all the audio features
scatter = sns.heatmap(features_df.corr(),  
annot=True,
linewidths=.5, cmap='RdPu')

In [ ]:
features_df.hist(figsize = (15,15))
plt.show()

### Recomendations using: 
- Gender, Fragrance_Family, Ingredents, Top_note, Middle_note, Base_note

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

# helper function to get the indices
def get_index(Name):
    return Dataset[Dataset.Name == Name].index[0]

#step1: combine features 
features_df = [
 'Gender',
 'Fragrance_Family',
 'Ingredients',
 'Top_note',
 'Middle_note',
 'Base_note']

for feature in features_df:
    Dataset[feature] = Dataset[feature].fillna('')

def comine_features(row):
    try:
        return row['Gender']+ " "+row['Fragrance_Family'] + " "+row['Ingredients']+" "+row['Ingredients'] + " "+row['Top_note'] + " "+row['Middle_note']+" "+row['Base_note']
    except:
        print("Error:", row)

Dataset['combined_features'] = Dataset.apply(comine_features, axis=1)

# step4: creat count matrix 
cv = CountVectorizer()
count_matrix = cv.fit_transform(Dataset['combined_features'])

# step5: cosine similarity 
cosine_sim = cosine_similarity(count_matrix)
perfume_test = "Yves Saint Laurent Libre" 

#step6: getting the index of the recommended perfume 
perfume_index = get_index(perfume_test)
similar_perfumes = list(enumerate(cosine_sim[perfume_index]))
sorted_list  = sorted(similar_perfumes, key=lambda x:x[1], reverse=True)
similar = sorted_list[1:11]
perfume_indecs = [i[0] for i in similar]

print(Dataset['Name'].iloc[perfume_indecs])

1738                             David Kasab Imperial EDP
2825    Yves Saint Laurent Libre Gift Box - (EDP 50 ml...
2826    Yves Saint Laurent Libre Gift Box - (EDP 50 ml...
868                                  Korloff Lady Intense
6                                Roberto Cavalli Florence
2795                  Yves Saint Laurent Black Opium Neon
3060                            Guerlain Shalimar Souffle
1872                  Al Dakheel Oud Bushra Cream - 50 ml
792                                   Mercedes Benz Woman
235                    Bvlgari Splendida Magnolia Sensuel
Name: Name, dtype: object


In [10]:
Dataset[Dataset.Name == "Yves Saint Laurent Libre"]

,Name,Price,Description,Rate,Rating_count,image,Brand,Gender,Product_Type,Character_x,Fragrance_Family,Size,Year,Ingredients,Concentration,Top_note,Middle_note,Base_note,combined_features
2,Yves Saint Laurent Libre,389,"This perfume is a reflection of Freedom, speci...",5,3 Rating,https://assets.goldenscent.com/catalog/product...,Yves Saint Laurent,Women,Perfume,Romantic,Floral,90 ml,2019,"Mandarin Orange, lavendar, black currant, peti...",Eau de Parfum,"Mandarin Orange, Lavendar, Black Currant, Peti...","Jasmine, Orange Blossom","Vanilla, Cedar, Musk, Ambergris","Women Floral Mandarin Orange, lavendar, black ..."
